## 1. Import packages

In [1]:
#!pip install tweepy

In [2]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm

## 2. Twitter API authentication

In [3]:
consumer_api_key = os.environ["TWITTER_CONSUMER_API_KEY"]
consumer_api_secret = os.environ["TWITTER_CONSUMER_API_SECRET"]

In [4]:
auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)

In [5]:
api = tw.API(auth, wait_on_rate_limit=True)

## 3. Tweets query

### 3.1. Define the query

In [6]:
search_words = "#covid19 -filter:retweets"
date_since = "2020-03-01"
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(7500)

### 3.2. Retreive the tweets

In [7]:
tweets_copy = []
for tweet in tqdm(tweets):
    tweets_copy.append(tweet)

7500it [11:03, 11.31it/s]


In [8]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 7500


## 4. Populate the dataset

In [9]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': tweet.text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|█████████████████████████████████████████████████████████████████████████████| 7500/7500 [00:42<00:00, 176.54it/s]


In [10]:
tweets_df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,Janci,,,2011-07-28 15:55:21,44,117,4401,False,2020-07-31 19:20:07,Outrageous!! The dipshittery amazes me why any...,"[CorruptGOP, GTFO]",Twitter for iPad,False
0,MNMommaof3,"Minnesota, USA","Mom of 3, Democrat and many other things. 😊",2020-05-14 15:17:11,15,88,364,False,2020-07-31 19:20:05,"Me: ""Ready to go to Wal-Mart!""\nTeenage Son: (...",None,Twitter for Android,False
0,All Things Travel,Worldwide,Online Travel Portal offering Special Deals fr...,2011-02-27 02:11:28,45133,24832,180,False,2020-07-31 19:20:05,If you are wondering why countries like #Croat...,"[Croatia, Slovenia, macedonia, greece, USA]",LinkedIn,False
0,Joshua Plante,"Berwick, ME","Facts matter, and I believe in them.",2014-02-19 18:47:39,175,405,9663,False,2020-07-31 19:20:05,@JoeBiden is the example to the nation about t...,[COVID19],Twitter for iPhone,False
0,JCDHE,"Olathe, KS","Mission: Protect the health and environment, p...",2009-06-23 18:36:49,3963,633,2638,True,2020-07-31 19:20:05,CHILD CARE PROVIDERS: Please check your provid...,None,Hootsuite Inc.,False


## 5. Save the data

### 5.1. Read past data

In [11]:
tweets_old_df = pd.read_csv("covid19_tweets.csv")
print(f"past tweets: {tweets_old_df.shape}")

past tweets: (44436, 13)


### 5.2. Merge past and present data

In [12]:
tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)
print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {tweets_all_df.shape[0]}")

new tweets: 7500 past tweets: 44436 all tweets: 51936


### 5.3. Drop duplicates

In [13]:
tweets_all_df.drop_duplicates(subset = ["user_name", "date", "text"], inplace=True)
print(f"all tweets: {tweets_all_df.shape}")

all tweets: (51936, 13)


### 5.4. Export the updated data

In [14]:
tweets_all_df.to_csv("covid19_tweets.csv", index=False)